## Pakiety

In [2]:
import numpy as np
import pandas as pd
import sklearn
import category_encoders
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

## Wczytanie danych

In [3]:
data=pd.read_csv("C://Users//Paulina//Downloads//mushroom-classification/mushrooms.csv")

In [4]:
data.head()

,class,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
0,p,x,s,n,t,p,f,c,n,k,...,s,w,w,p,w,o,p,k,s,u
1,e,x,s,y,t,a,f,c,b,k,...,s,w,w,p,w,o,p,n,n,g
2,e,b,s,w,t,l,f,c,b,n,...,s,w,w,p,w,o,p,n,n,m
3,p,x,y,w,t,p,f,c,n,n,...,s,w,w,p,w,o,p,k,s,u
4,e,x,s,g,f,n,f,w,b,k,...,s,w,w,p,w,o,e,n,a,g


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8124 entries, 0 to 8123
Data columns (total 23 columns):
class                       8124 non-null object
cap-shape                   8124 non-null object
cap-surface                 8124 non-null object
cap-color                   8124 non-null object
bruises                     8124 non-null object
odor                        8124 non-null object
gill-attachment             8124 non-null object
gill-spacing                8124 non-null object
gill-size                   8124 non-null object
gill-color                  8124 non-null object
stalk-shape                 8124 non-null object
stalk-root                  8124 non-null object
stalk-surface-above-ring    8124 non-null object
stalk-surface-below-ring    8124 non-null object
stalk-color-above-ring      8124 non-null object
stalk-color-below-ring      8124 non-null object
veil-type                   8124 non-null object
veil-color                  8124 non-null object
ring-number

### widać, że w danych nie ma braków, są tylko zmienne typu object

## Zakodowanie klasy na 0/1

In [6]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y=le.fit_transform(data['class'])
data.drop('class', axis=1, inplace=True)

## Podział na train/test

In [8]:
from sklearn.model_selection import train_test_split
seed = 70
test_size = 0.2
x_train, x_test, y_train, y_test = train_test_split(data, y, test_size=test_size, random_state=seed)

## Target encoding 

In [11]:
te=category_encoders.target_encoder.TargetEncoder(x_train)
x_train=x_train.apply(lambda col: te.fit_transform(col.values, y_train, smoothing=10).squeeze()) 

In [12]:
x_test=x_test.apply(lambda col: te.transform(col.values).squeeze()) 

# Klasyfikatory

- Regresja logistyczna
- RandomForest
- xgboost
- SVM

In [13]:
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

def classifier(model):
    model.fit(x_train, y_train)
    accuracy=accuracy_score(model.predict(x_test),y_test)
    return accuracy*100

In [20]:
print("Accuracy Logistic Regression: %.2f%%" % (classifier(LogisticRegression())))

Accuracy Logistic Regression: 50.03%


In [22]:
print("Accuracy RandomForest: %.2f%%" % (classifier(RandomForestClassifier(max_depth=6, random_state=8))))

Accuracy RandomForest: 46.83%


In [23]:
print("Accuracy XGBoost: %.2f%%" % (classifier(XGBClassifier())))

Accuracy XGBoost: 50.40%


In [24]:
print("Accuracy SVM: %.2f%%" % (classifier(SVC())))

Accuracy SVM: 41.66%


# Zmiana sposoby kodowania zmiennych

### Uzyskane wyniki uzyskane nie są zadowalające (dotyczy to wszystkich klasyfikatorów). Zamiast skupić się na strojeniu hiperparametrów, postanowiono zmienić sposób zakodowania zmiennych kategorycznych z target encoding na one-hot.

In [25]:
data_onehot=pd.get_dummies(data)

 W przypadku używania kodowania one-hot można w prosty sposób skorzystać z kroswalidacji (gdy używa się target encoding należy napisać samemu pętlę kroswalidacyjną

In [38]:
from sklearn.model_selection import cross_val_score
def classifier_cv(model):
    res=cross_val_score(model, data_onehot, y, scoring='accuracy', cv = 10)
    return (np.mean(res*100), np.std (res*100))

In [43]:
classifier_cv (LogisticRegression())

(96.77588327607414, 6.6331245147083076)

In [40]:
classifier_cv (RandomForestClassifier(max_depth=6, random_state=8))

(95.97679942316664, 9.272641776947637)

In [41]:
classifier_cv (XGBClassifier())

(96.75264634419743, 9.4182353881735)

In [42]:
classifier_cv(SVC())

(95.84131629493635, 9.594517301875863)

# Podsumowanie

 Uzyskano lepsze klasyfikatory gdy zrezygnowano z target encoding na rzecz one-hot encoding. Cztery analizowane klasyfikatory dały wynik wówczas podobny średni wynik - ok. **96%**, jednak najmniejszym odchyleniem standardowym charakteryzował się klasyfikator w oparciu o **regresję logistyczną**.